In [1]:
import re
import string
from nltk.stem import WordNetLemmatizer
from wordcloud import STOPWORDS
from sklearn.model_selection import train_test_split
from functools import reduce

In [55]:
class Classificator:

    def __init__(self):
        self.vocab = set()
        self.count = {
            'spam': {},
            'ham': {}
        }
        self.freq = {
            'spam': {},
            'ham': {}
        }
    
    @classmethod
    def get_vocab(cls, X):
        vocab = map(lambda x: set(x), X)
        vocab = reduce(lambda x1, x2: x1.union(x2), vocab)
        return vocab
    
    @classmethod
    def word_count(cls, X, Y, count=None):
        if count is None:
            count = {
                'spam': {},
                'ham': {}
            }
        for x, y in zip(X, Y):
            for word in x:
                if not word in count[y].keys():
                    count[y][word] = 1
                else:
                    count[y][word] += 1
        return count
    
    def word_freq(self, count, vocab):
        freq = {
            'spam': {},
            'ham': {}
        }
        Nclass = {
            'spam': reduce(lambda w1, w2: w1 + w2, count['spam'].values()),
            'ham': reduce(lambda w1, w2: w1 + w2, count['ham'].values())
        }
        V = len(vocab)
        for y, ycount in count.items():
            for word, _count in ycount.items():
                freq[y][word] = (_count + 1) / (Nclass[y] + V)
        return freq
    
    def train(self, X, Y):
        self.vocab.update(self.get_vocab(X))
        self.word_count(X, Y, self.count)
        self.freq = self.word_freq(self.count, self.vocab)
    
    def predict(self, X):
        assert isinstance(X, list) and len(X) != 0
        single = isinstance(X[0], str)
        if single:
            X = [X]
        Y = []
        for x in X:
            y_value = 1
            for word in x:
                if word in self.freq['spam'].keys() and word in self.freq['ham'].keys():
                    y_value *= self.freq['spam'][word] / self.freq['ham'][word]
            y_value = 'spam' if y_value > 1 else 'ham'
            Y.append(y_value)
        if single:
            Y = Y[0]
        return Y
    
    def test(self, X, Y):
        pos = 'spam'
        neg = 'ham'
        Y_predicted = self.predict(X)
        TP, TN, FP, FN = 0, 0, 0, 0
        for y, y_p in zip(Y, Y_predicted):
            if y_p == pos:
                if y_p == y: TP += 1
                else: FP += 1
            else:
                if y_p == y: TN += 1
                else: FN += 1
        accuracy = self.accuracy(TP, TN, FP, FN)
        recall = self.recall(TP, FN)
        precision = self.precision(TP, FP)
        f1 = self.f1(recall, precision)
        return accuracy, recall, precision, f1

    @classmethod
    def accuracy(cls, TP, TN, FP, FN):
        return (TP+TN)/(TP+FP+FN+TN)

    @classmethod
    def recall(cls, TP, FN):
        return TP/(TP+FN)

    @classmethod
    def precision(cls, TP, FP):
        return TP/(TP+FP)

    @classmethod
    def f1(cls, recall, precision):
        return 2*(recall*precision)/(recall+precision)

In [74]:
class Model:

    def __init__(self):
        self.classificator = Classificator()
    
    @classmethod
    def load(cls, filename):
        msgs = {
            'data' : [],
            'target' : []
        }

        with open(filename) as f:
            lines = f.readlines()
            for line in lines:
                target, msg = line.split('\t')
                msgs['data'].append(msg)
                msgs['target'].append(target)
        
        return msgs

    @classmethod
    def preprocess(cls, X):
        # lowercase (str -> str)
        X = [line.lower() for line in X]

        # numbers (str -> str)
        X = [re.sub(r'\d+', '', line) for line in X]

        # punctuation (str -> str)
        X = [line.translate(str.maketrans("", "", string.punctuation)) for line in X]

        #replace ’ with ' and remove “ and ” (str -> str)
        X = [line.replace("’","'") for line in X]
        X = [line.replace("“","") for line in X]
        X = [line.replace("”","") for line in X]

        # whitespaces (str -> str)
        X = [line.strip() for line in X]

        # tokenization (str -> list)
        X = [line.split(' ') for line in X]

        # stopwords (list -> list)
        # stopwords = set(STOPWORDS)
        # stopwords.update({})
        # X = [[word for word in line if len(word) > 2 and not word in stopwords] for line in X]

        # Lemmatization (list -> list)
        X = [[WordNetLemmatizer().lemmatize(word) for word in line] for line in X]

        return X
    
    @classmethod
    def split(cls, data, target, test_size=0.3, random_state=109):
        # Split dataset into training set and test set
        # 70% training and 30% test (default)
        return train_test_split(data, target, test_size=test_size, random_state=random_state)
    
    def train_test(self, filename="SMSSpamCollection", test_size=0.3, random_state=109):
        msgs = self.load(filename)
        data = self.preprocess(msgs['data'])
        X_train, X_test, Y_train, Y_test = self.split(data, msgs['target'], test_size, random_state)
        self.classificator.train(X_train, Y_train)
        return self.classificator.test(X_test, Y_test)
    
    def predict(self, X):
        if isinstance(X, str):
            X = [X]
        data = self.preprocess(X)
        return self.classificator.predict(data)

In [75]:
model = Model()
model.train_test()

(0.9533771667662881, 0.853448275862069, 0.8181818181818182, 0.8354430379746834)

In [65]:
model.predict("I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.")

['ham']

In [70]:
model.predict("Congrats! Urgent! 1 year special cinema pass for 2 is yours.")

['spam']